In [1]:
import xarray as xr
import numpy as np
import zarr
from datetime import datetime, timezone
import BuildZarrStore as bzs
import pandas as pd
import os

In [ ]:
var ="RR"


folder_path = f'INCA_data/{var}'

filepaths = []

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        filepaths.append(file_path)

In [ ]:
store = zarr.open(f'INCA.zarr/{var}', mode='r')
arr = store[f'{var}']
dtype=arr.dtype
fill_value = arr.attrs.get('_FillValue', None)

store = zarr.storage.LocalStore("INCA.zarr")
group = zarr.group(store=store)[var]
x_extent = group["x"][:]
y_extent = group["y"][:]

for i, file in enumerate(filepaths):
    data = xr.open_dataset(file, chunks={}, mask_and_scale=False)
    data = data.load()

    x_min, x_max = bzs.get_idx(x_extent, data["x"].values)
    y_min, y_max = bzs.get_idx(y_extent, data["y"].values)

    origin = np.datetime64("2011-03-15T00:00:00").astype("datetime64[h]")
    time_min, time_max = data.time.values[0].astype("datetime64[h]"), data.time.values[-1].astype("datetime64[h]")+1
    time_delta_min, time_delta_max = (time_min - origin).astype("int64"), (time_max - origin).astype("int64")

    full_range = pd.date_range(time_min, time_max, freq="1H").values.astype("datetime64[ns]")

    for value in data.time.values:
        if value in set(full_range):
            continue
        else:
            print(f"{file} Data incomplete")
            empty_array = np.full((full_range.shape[0], data["x"].values.shape[0], data["y"].values.shape[0]),
                                fill_value=fill_value, dtype=dtype)

            template = xr.Dataset({f"{var}": (("time", "x", "y"), empty_array)},
                                  coords={
                                    "time": full_range,
                                    "x": data["x"].values,
                                    "y": data["y"].values
                                  }
                                  )

            data_filled = data.combine_first(template)
            print(f"{file} Data gaps filled with no data values")
            break


    group[var][time_delta_min:time_delta_max, y_min:y_max, x_min:x_max] = data[var].values

    print(f"{file} written to zarr store. {i}/{len(filepaths)} complete💌")
    

INCA_RR_data/INCAL_HOURLY_RR_202107.nc written to zarr store.
INCA_RR_data/INCAL_HOURLY_RR_201502.nc written to zarr store.
INCA_RR_data/INCAL_HOURLY_RR_201304.nc written to zarr store.
INCA_RR_data/INCAL_HOURLY_RR_202308.nc written to zarr store.
INCA_RR_data/INCAL_HOURLY_RR_201111.nc written to zarr store.
INCA_RR_data/INCAL_HOURLY_RR_201808.nc written to zarr store.
INCA_RR_data/INCAL_HOURLY_RR_202503.nc written to zarr store.
INCA_RR_data/INCAL_HOURLY_RR_202101.nc written to zarr store.
INCA_RR_data/INCAL_HOURLY_RR_201308.nc written to zarr store.
INCA_RR_data/INCAL_HOURLY_RR_202103.nc written to zarr store.
INCA_RR_data/INCAL_HOURLY_RR_202008.nc written to zarr store.
INCA_RR_data/INCAL_HOURLY_RR_202001.nc written to zarr store.
INCA_RR_data/INCAL_HOURLY_RR_201108.nc written to zarr store.
INCA_RR_data/INCAL_HOURLY_RR_201305.nc written to zarr store.
INCA_RR_data/INCAL_HOURLY_RR_202004.nc written to zarr store.
INCA_RR_data/INCAL_HOURLY_RR_201802.nc written to zarr store.
INCA_RR_

### Inspecting the zarr store

In [3]:
ds = xr.open_zarr("INCA.zarr", group="RR", consolidated=True, chunks={})#, decode_times=False)
ds

<xarray.Dataset> Size: 450GB
Dimensions:  (x: 701, time: 200000, y: 401)
Coordinates:
  * x        (x) float64 6kB 2e+04 2.1e+04 2.2e+04 ... 7.18e+05 7.19e+05 7.2e+05
  * time     (time) datetime64[ns] 2MB 2011-03-15 ... 2034-01-06T07:00:00
  * y        (y) float64 3kB 2.2e+05 2.21e+05 2.22e+05 ... 6.19e+05 6.2e+05
Data variables:
    RR       (time, y, x) float64 450GB dask.array<chunksize=(720, 100, 100), meta=np.ndarray>
Attributes: (12/13)
    Description:         1-hour precipitation sum
    Unit:                kg m-2
    name:                incal-hourly
    Conventions:         CF-1.7
    freq:                1H
    spatial_resolution:  1000
    ...                  ...
    Institution:         GeoSphere Austria
    history:             Original data produced by GeoSphere Austria
    source:              modeled data
    crs:                 EPSG: 31287
    grid_mapping:        lambert_conformal_conic
    Version:             1

In [7]:
filtered = ds.sel(time=slice("2019-06-01T00:00:00.000000000","2019-06-30T23:00:00.000000000"))

In [8]:
loaded = filtered.load()
loaded

<xarray.Dataset> Size: 2GB
Dimensions:  (x: 701, time: 720, y: 401)
Coordinates:
  * x        (x) float64 6kB 2e+04 2.1e+04 2.2e+04 ... 7.18e+05 7.19e+05 7.2e+05
  * time     (time) datetime64[ns] 6kB 2019-06-01 ... 2019-06-30T23:00:00
  * y        (y) float64 3kB 2.2e+05 2.21e+05 2.22e+05 ... 6.19e+05 6.2e+05
Data variables:
    RR       (time, y, x) float64 2GB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes: (12/13)
    Description:         1-hour precipitation sum
    Unit:                kg m-2
    name:                incal-hourly
    Conventions:         CF-1.7
    freq:                1H
    spatial_resolution:  1000
    ...                  ...
    Institution:         GeoSphere Austria
    history:             Original data produced by GeoSphere Austria
    source:              modeled data
    crs:                 EPSG: 31287
    grid_mapping:        lambert_conformal_conic
    Version:             1

In [2]:
xr.open_dataset("INCA_data/INCAL_HOURLY_RR_201906.nc").load()

<xarray.Dataset> Size: 2GB
Dimensions:                  (time: 720, y: 401, x: 701)
Coordinates:
    lambert_conformal_conic  float64 8B nan
    lat                      (y, x) float32 1MB 45.77 45.77 45.77 ... 49.4 49.4
    lon                      (y, x) float32 1MB 8.445 8.457 8.47 ... 17.73 17.74
  * time                     (time) datetime64[ns] 6kB 2019-06-01 ... 2019-06...
  * x                        (x) int32 3kB 20000 21000 22000 ... 719000 720000
  * y                        (y) int32 2kB 220000 221000 ... 619000 620000
Data variables:
    RR                       (time, y, x) float64 2GB 0.0 0.0 0.0 ... 0.0 0.0
Attributes: (12/14)
    Conventions:         CF-1.7
    author:              Alexander Kann (alexander.kann@zamg.ac.at), Benedikt...
    comment:             No additional comments
    crs:                 EPSG:31287
    freq:                1H
    grid_mapping:        lambert_conformal_conic
    ...                  ...
    name:                incal-hourly
    references:          https://www.zamg.ac.at/cms/de/forschung/wetter/inca ...
    source:              modeled data
    spatial_domain:      INCAL
    spatial_resolution:  1000
    title:               INCA analysis - large domain